<a href="https://colab.research.google.com/github/Viny2030/HUMAI/blob/main/NLPDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Construcción de un Dataset para Procesamiento de Lenguaje Natural

En este notebook vamos a aprender a trabajar con texto para generar un dataset que nos permita entrenar modelos de Procesamiento de Lenguaje Natural. Para esto vamos a ver un ejemplo en una tarea particular, pero los conceptos necesarios para construir este dataset son aplicables a todas las tareas del rubro.  

## Inferencia del lenguaje natural

La **Inferencia del lenguaje natural** estudia si una *hipótesis*
se puede inferir de una *premisa*, donde ambas son una secuencia de texto.
En otras palabras, la inferencia del lenguaje natural determina la relación lógica entre un par de secuencias de texto.
Estas relaciones suelen clasificarse en tres tipos:

* *Implicación* (entailment): la hipótesis se puede inferir de la premisa.
* *Contradicción* (contradiction): la negación de la hipótesis se puede inferir de la premisa.
* *Neutral*: todos los demás casos.

La inferencia del lenguaje natural también se conoce como tarea de reconocimiento de vinculación textual.

Por ejemplo, el siguiente par se etiquetará como *Implicación* porque  "showing affection" en la hipótesis se puede inferir de "hugging one another" en la premisa.

> Premise: Two women are hugging each other.

> Hypothesis: Two women are showing affection.

El siguiente es un ejemplo de *contradicción* ya que "running the coding example" indica "not sleeping" en lugar de "sleeping".

> Premise: A man is running the coding example from Dive into Deep Learning.

> Hypothesis: The man is sleeping.

El tercer ejemplo muestra una relación de *neutralidad* porque ni "famous" ni "not famous" pueden inferirse del hecho de que "are performing for us".

> Premise: The musicians are performing for us.

> Hypothesis: The musicians are famous.

La inferencia del lenguaje natural ha sido un tema central para comprender el lenguaje natural. Disfruta de amplias aplicaciones que van desde la recuperación de información hasta la respuesta a preguntas de dominio abierto. Para estudiar este problema, comenzaremos investigando un conjunto de datos de referencia de inferencia de lenguaje natural popular.




## The Stanford Natural Language Inference (SNLI) Dataset

El corpus de inferencia del lenguaje natural de Stanford (SNLI) es una colección de más de 500 000 pares de oraciones etiquetadas en inglés.
Descargamos y almacenamos el conjunto de datos SNLI extraído en la ruta `../data/snli_1.0`.

In [ ]:
%%capture
!pip install d2l==1.0.3

In [ ]:
import os
import re
import torch
from torch import nn
from d2l import torch as d2l

d2l.DATA_HUB['SNLI'] = (
    'https://nlp.stanford.edu/projects/snli/snli_1.0.zip',
    '9fcde07509c7e87ec61c640c1b2753d9041758e4')

data_dir = d2l.download_extract('SNLI')

El dataset está estructurado como un archivo separado por tabs. Usaremos pandas para leerlo.

In [ ]:
import pandas as pd
df = pd.read_csv(os.path.join(data_dir, 'snli_1.0_train.txt'), sep='\t')
df.head()

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5
0,neutral,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,3416050480.jpg#4,3416050480.jpg#4r1n,neutral,NaN,NaN,NaN,NaN
1,contradiction,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",3416050480.jpg#4,3416050480.jpg#4r1c,contradiction,NaN,NaN,NaN,NaN
2,entailment,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,"( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",3416050480.jpg#4,3416050480.jpg#4r1e,entailment,NaN,NaN,NaN,NaN
3,neutral,( Children ( ( ( smiling and ) waving ) ( at c...,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,Children smiling and waving at camera,They are smiling at their parents,2267923837.jpg#2,2267923837.jpg#2r1n,neutral,NaN,NaN,NaN,NaN
4,entailment,( Children ( ( ( smiling and ) waving ) ( at c...,( There ( ( are children ) present ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,Children smiling and waving at camera,There are children present,2267923837.jpg#2,2267923837.jpg#2r1e,entailment,NaN,NaN,NaN,NaN


### Leyendo el dataset

El conjunto de datos SNLI original contiene información mucho más rica de la que realmente necesitamos en nuestros experimentos. Por lo tanto, definimos una función `read_snli` para extraer solo parte del conjunto de datos y luego devolver listas de premisas, hipótesis y sus etiquetas.


In [ ]:
def read_snli(data_dir, is_train):
    """Read the SNLI dataset into premises, hypotheses, and labels."""
    def extract_text(s):
        # Remove information that will not be used by us
        s = re.sub('\\(', '', s)
        s = re.sub('\\)', '', s)
        # Substitute two or more consecutive whitespace with space
        s = re.sub('\\s{2,}', ' ', s)
        return s.strip()
    label_set = {'entailment': 0, 'contradiction': 1, 'neutral': 2}
    file_name = os.path.join(data_dir, 'snli_1.0_train.txt'
                             if is_train else 'snli_1.0_test.txt')
    with open(file_name, 'r') as f:
        rows = [row.split('\t') for row in f.readlines()[1:]]
    premises = [extract_text(row[1]) for row in rows if row[0] in label_set]
    hypotheses = [extract_text(row[2]) for row in rows if row[0] in label_set]
    labels = [label_set[row[0]] for row in rows if row[0] in label_set]
    return premises, hypotheses, labels

Ahora imprimamos los primeros 3 pares de premisa e hipótesis, así como sus etiquetas ("0", "1" y "2" corresponden a "entailment", "contradiction" y "neutral", respectivamente).


In [ ]:
train_data = read_snli(data_dir, is_train=True)
for x0, x1, y in zip(train_data[0][:3], train_data[1][:3], train_data[2][:3]):
    print('premise:', x0)
    print('hypothesis:', x1)
    print('label:', y)

premise: A person on a horse jumps over a broken down airplane .
hypothesis: A person is training his horse for a competition .
label: 2
premise: A person on a horse jumps over a broken down airplane .
hypothesis: A person is at a diner , ordering an omelette .
label: 1
premise: A person on a horse jumps over a broken down airplane .
hypothesis: A person is outdoors , on a horse .
label: 0


El conjunto de entrenamiento tiene alrededor de 550000 pares,
y el conjunto de prueba tiene alrededor de 10000 pares.
A continuación se muestra que
las tres etiquetas están equilibradas en
tanto el conjunto de entrenamiento como el de prueba.

In [ ]:
test_data = read_snli(data_dir, is_train=False)
for data in [train_data, test_data]:
    print([[row for row in data[2]].count(i) for i in range(3)])

[183416, 183187, 182764]
[3368, 3237, 3219]


### Tokenización con Spacy

La tokenización es la tarea de dividir un texto en segmentos significativos, llamados tokens. La entrada al tokenizador es un texto Unicode y la salida es un objeto Doc de Spacy.

La tokenización de spaCy no es destructiva, lo que significa que siempre podrás reconstruir la entrada original a partir de la salida tokenizada. La información de los espacios en blanco se conserva en los tokens y no se agrega ni elimina ninguna información durante la tokenización. Este es una especie de principio básico del objeto Doc de spaCy: `doc.text == input_text` siempre debe ser verdadero.


Durante el procesamiento, spaCy primero tokeniza el texto, es decir, lo segmenta en palabras, signos de puntuación, etc. Esto se hace aplicando reglas específicas de cada idioma. Por ejemplo, la puntuación al final de una frase debe separarse, mientras que “EE.UU.” debería seguir siendo un sólo token.

Acontinuación vamos a descargar los modelos para los idiomas Español e Inglés.

In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 47.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy

eng_spacy = spacy.load("en_core_web_sm") # Cargue el modelo en inglés para tokenizar el texto en inglés
def engTokenize(text):
    """
    Tokeniza un texto en inglés y devuelve una lista de tokens
    """
    return [token.text for token in eng_spacy.tokenizer(text)]


In [ ]:
[engTokenize(x) for x in train_data[1][:6]]

[['A',
  'person',
  'is',
  'training',
  'his',
  'horse',
  'for',
  'a',
  'competition',
  '.'],
 ['A',
  'person',
  'is',
  'at',
  'a',
  'diner',
  ',',
  'ordering',
  'an',
  'omelette',
  '.'],
 ['A', 'person', 'is', 'outdoors', ',', 'on', 'a', 'horse', '.'],
 ['They', 'are', 'smiling', 'at', 'their', 'parents'],
 ['There', 'are', 'children', 'present'],
 ['The', 'kids', 'are', 'frowning']]

In [ ]:
engTokenize("AFK is an abbreviation for 'away from keyboard'.")

['AFK',
 'is',
 'an',
 'abbreviation',
 'for',
 "'",
 'away',
 'from',
 'keyboard',
 "'",
 '.']

### Creación del Vocabulario

Estos tokens siguen siendo cadenas. Sin embargo, las entradas a nuestros modelos deben consistir en última instancia en entradas numéricas. A continuación, presentamos una clase para construir **`vocabularios`**, es decir, objetos que asocian cada valor de token distinto con un índice único. Primero, determinamos el conjunto de tokens únicos en nuestro corpus de entrenamiento. Luego asignamos un índice numérico a cada token único. Los elementos raros del vocabulario a menudo se eliminan por conveniencia. Cada vez que nos encontramos con un token en el tiempo de entrenamiento o prueba que no se había visto previamente o se había eliminado del vocabulario, la representamos por un token especial **`“<unk>”`**, lo que significa que este es un valor desconocido.

In [ ]:
import collections

class Vocab:
    """Vocabulario para texto."""
    def __init__(self, tokens=[], min_freq=0, reserved_tokens=[]):
        """Inicializa el vocabulario."""
        # Aplana una lista 2D si es necesario
        if tokens and isinstance(tokens[0], list):
            tokens = [token for line in tokens for token in line]
        # Cuenta las frecuencias de los tokens
        counter = collections.Counter(tokens)
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                  reverse=True)
        # La lista de tokens únicos
        self.idx_to_token = list(sorted(set(['<unk>'] + reserved_tokens + [
            token for token, freq in self.token_freqs if freq >= min_freq])))
        # Mapea cada token a su índice
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}

    def __len__(self):
        # Retorna el tamaño del vocabulario
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        # Retorna el índice de un token o una lista de índices para varios tokens
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        # Convierte un índice o una lista de índices en sus tokens correspondientes
        if hasattr(indices, '__len__') and len(indices) > 1:
            return [self.idx_to_token[int(index)] for index in indices]
        return self.idx_to_token[indices]

    @property
    def unk(self):  # Índice para el token desconocido
        return self.token_to_idx['<unk>']



### Definición de una clase para cargar el conjunto de datos

A continuación definimos una clase para cargar el conjunto de datos SNLI heredando de la clase `Dataset`. El argumento `num_steps` en el constructor de clases especifica la longitud de una secuencia de texto para que cada minilote de secuencias tenga la misma forma.
En otras palabras, los tokens después de los primeros `num_steps` en una secuencia más larga se recortan, mientras que los tokens especiales “&lt;pad&gt;” se agregarán a secuencias más cortas hasta que su longitud se convierta en "num_steps".
Al implementar la función `__getitem__`, podemos acceder arbitrariamente a la premisa, hipótesis y etiqueta con el índice `idx`.

In [ ]:
class SNLIDataset(torch.utils.data.Dataset):
    """Un dataset personalizado para cargar el conjunto de datos SNLI."""

    def __init__(self, dataset, num_steps, tokenizer_function, vocab=None):
        """Inicializa el dataset con el conjunto de datos SNLI."""
        self.num_steps = num_steps
        all_premise_tokens = [tokenizer_function(x) for x in dataset[0]]
        all_hypothesis_tokens = [tokenizer_function(x) for x in dataset[1]]
        if vocab is None:
            self.vocab = Vocab(all_premise_tokens + all_hypothesis_tokens,
                               min_freq=1, reserved_tokens=['<pad>'])
        else:
            self.vocab = vocab
        self.premises = self._pad(all_premise_tokens)
        self.hypotheses = self._pad(all_hypothesis_tokens)
        self.labels = torch.tensor(dataset[2])
        print('Se leyeron ' + str(len(self.premises)) + ' ejemplos')

    def _pad(self, lines):
        """Realiza el padding o truncado de las secuencias a una longitud fija."""
        def truncate_pad(line, num_steps, padding_token):
            if len(line) > num_steps:
                return line[:num_steps]
            return line + [padding_token] * (num_steps - len(line))

        return torch.tensor([
            truncate_pad(self.vocab[line], self.num_steps, self.vocab['<pad>'])
            for line in lines])

    def __getitem__(self, idx):
        """Retorna un ejemplo en el índice dado."""
        return (self.premises[idx], self.hypotheses[idx]), self.labels[idx]

    def __len__(self):
        """Retorna el número de ejemplos en el dataset."""
        return len(self.premises)


### Juntando todo

Ahora podemos invocar la función `read_snli` y la clase `SNLIDataset` para descargar el conjunto de datos SNLI y devolver instancias de `DataLoader` para los conjuntos de entrenamiento y prueba, junto con el vocabulario del conjunto de entrenamiento.
Es de destacar que debemos utilizar el vocabulario construido a partir del conjunto de entrenamiento como el del conjunto de prueba.
Como resultado, cualquier token nuevo del conjunto de prueba será desconocido para el modelo entrenado en el conjunto de entrenamiento.

In [ ]:
def load_data_snli(batch_size, tokenizer_function, num_steps=50):
    """Download the SNLI dataset and return data iterators and vocabulary."""
    num_workers = 2
    data_dir = d2l.download_extract('SNLI')
    train_data = read_snli(data_dir, True)
    test_data = read_snli(data_dir, False)
    train_set = SNLIDataset(train_data, num_steps, tokenizer_function)
    test_set = SNLIDataset(test_data, num_steps, tokenizer_function, train_set.vocab)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size,
                                             shuffle=True,
                                             num_workers=num_workers)
    test_iter = torch.utils.data.DataLoader(test_set, batch_size,
                                            shuffle=False,
                                            num_workers=num_workers)
    return train_iter, test_iter, train_set.vocab

Aquí configuramos el tamaño del lote en 128 y la longitud de la secuencia en 50, e invocamos la función `load_data_snli` para obtener los iteradores de datos y el vocabulario. Luego imprimimos el tamaño del vocabulario.


In [ ]:
eng_spacy = spacy.load("en_core_web_sm")
def engTokenize(text): return [token.text for token in eng_spacy.tokenizer(text)]
train_iter, test_iter, vocab = load_data_snli(128, engTokenize, 50)
len(vocab)

Se leyeron 549367 ejemplos
Se leyeron 9824 ejemplos


39342

Ahora imprimimos la forma del primer minibatch. Tenemos dos entradas `X[0]` y `X[1]` que representan pares de premisas e hipótesis.


In [ ]:
for X, Y in train_iter:
    print(X[0].shape)
    print(X[1].shape)
    print(Y.shape)
    break

torch.Size([128, 50])
torch.Size([128, 50])
torch.Size([128])
